### Clustering Analysis

- loads in previously generated dataset containing standardized gene expression levels for the top 5,000 varaible genes and top 10 sample sizes for tissue type
    - saved as "standardized_gtex_data.csv"

- Utilizes two clustering algorithms
    - 1. DBSCAN
    - 2. Spectral clustering


author: @emilyekstrum <br>
11/29/25

In [1]:
# load in modules
import pandas as pd
import numpy as np

### Load in saved standardized dataframe

In [4]:
standardized_df = pd.read_csv('standardized_gtex_data.csv', index_col=0)
standardized_df.head()

,ENSG00000244734.3,ENSG00000210082.2,ENSG00000198804.2,ENSG00000198712.1,ENSG00000198938.2,ENSG00000188536.12,ENSG00000198899.2,ENSG00000198886.2,ENSG00000275896.5,ENSG00000163220.10,...,ENSG00000176749.8,ENSG00000241973.10,ENSG00000118689.14,ENSG00000167680.15,ENSG00000148053.15,ENSG00000134291.11,ENSG00000183578.6,ENSG00000164091.11,ENSG00000173418.11,Tissue
SampleID,,,,,,,,,,,,,,,,,,,,,
GTEX-1117F-0226-SM-5GZZ7,-0.189282,-0.638404,-0.912095,-1.145540,-0.795922,-0.192404,-1.089657,-1.153692,-0.132456,-0.248378,...,-0.362862,-0.284106,0.985051,0.126768,0.062134,-0.841136,0.994222,0.596135,0.676412,Adipose Tissue
GTEX-1117F-0426-SM-5EGHI,-0.192726,-0.486328,0.055921,0.109423,1.286259,-0.194876,0.890270,-0.001923,-0.129289,-0.248983,...,-0.376647,-0.951475,2.825548,-0.431997,-0.617946,-1.181526,-0.709254,-0.593598,1.388128,Muscle
GTEX-1117F-0526-SM-5EGHJ,-0.192060,-0.646184,-0.958799,-1.119740,-0.817392,-0.194339,-0.964504,-1.078079,-0.121652,-0.249558,...,-0.377515,-0.353600,1.399758,0.463296,-0.000186,-0.240959,2.179139,0.063419,0.096230,Blood Vessel
GTEX-1117F-0626-SM-5N9CS,-0.116158,-0.416350,-1.188713,-1.198601,-0.964272,-0.132521,-0.960315,-1.175524,-0.132202,-0.234583,...,-0.367923,-0.006885,0.918577,0.331479,0.178465,0.717285,0.703752,0.521705,0.288616,Blood Vessel
GTEX-1117F-0726-SM-5GIEN,-0.171616,0.695877,1.035389,0.193153,1.388733,-0.177632,2.506260,1.878821,-0.131832,-0.253328,...,-0.415738,-0.897088,-0.601636,-0.270947,-0.458297,-0.623199,-0.521393,-1.110068,-0.814676,Heart


# Clustering

### Method 1: DBSCAN

### Method 2: Spectral Clustering